In [2]:
import pandas as pd
df = pd.read_csv("studentPerformance.csv")
y = df['Pass_Fail']
x = df.drop('Pass_Fail', axis=1)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train, y_train)
y_lr_train_pred = lr.predict(x_train)
y_lr_test_pred = lr.predict(x_test)
from sklearn.metrics import mean_squared_error , r2_score
lr_train_mse = mean_squared_error(y_train, y_lr_train_pred)
lr_train_r2 = r2_score(y_train, y_lr_train_pred)
lr_test_mse = mean_squared_error(y_test, y_lr_test_pred)
lr_test_r2 = r2_score(y_test, y_lr_test_pred)
lr_results = pd.DataFrame(['Linear regression', lr_train_mse, lr_train_r2, lr_test_mse, lr_test_r2]).transpose()
lr_results.columns = ['Method', 'Training MSE', 'Training R2', 'Test MSE', 'Test R2']

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_logreg_train_pred = logreg.predict(x_train)
y_logreg_test_pred = logreg.predict(x_test)
from sklearn.metrics import mean_squared_error , r2_score
logreg_train_mse = mean_squared_error(y_train, y_logreg_train_pred)
logreg_train_r2 = r2_score(y_train, y_logreg_train_pred)
logreg_test_mse = mean_squared_error(y_test, y_logreg_test_pred)
logreg_test_r2 = r2_score(y_test, y_logreg_test_pred)
logreg_results = pd.DataFrame(['Logistic regression', logreg_train_mse, logreg_train_r2, logreg_test_mse, logreg_test_r2]).transpose()
logreg_results.columns = ['Method', 'Training MSE', 'Training R2', 'Test MSE', 'Test R2']
df_models = pd.concat([lr_results, logreg_results], axis=0)
df_models.reset_index(drop=True)


import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(5,5))
plt.scatter(y_train, y_lr_train_pred)
z= np.polyfit(y_train, y_lr_train_pred, 1)
p = np.poly1d(z)
plt.plot(y_train,p(y_train),'#F8766D')
plt.scatter(y_train, y_logreg_train_pred)
c= np.polyfit(y_train, y_logreg_train_pred, 1)
p = np.poly1d(c)
plt.plot(y_train,p(y_train),'#F8766D')
plt.show()
df_models
import gradio as gr
def predict_score(study_hours, attendance, practice_tests):
    input_data = pd.DataFrame({
        'Study_Hours': [study_hours],
        'Attendance': [attendance],
        'Practice_Tests': [practice_tests]
    })
    predicted_score = lr.predict(input_data)[0]
    result = "Pass " if predicted_score >= 50 else "Fail "
    return {
        "Predicted Score": round(predicted_score, 2),
        "Result": result
    }
interface = gr.Interface(
    fn=predict_score,
    inputs=[gr.Slider(0, 10, step=0.1, label="Study Hours"),gr.Slider(50, 100, step=1, label="Attendance (%)"),gr.Number(label="Practice Tests", value=0)],
    outputs=[
        gr.Label(label="Predicted Score"),
        gr.Label(label="Result (Pass/Fail)")
    ],
    title="Student Performance Predictor",
    description="Entrez les caractéristiques d'un étudiant pour prédire son score final et s'il réussit ou échoue."
)

FileNotFoundError: [Errno 2] No such file or directory: 'studentPerformance.csv'